In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
import matplotlib.dates as plt_date
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Perceptron
import PIL


from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    f1_score, confusion_matrix

from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from xml.sax.saxutils import escape
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

from datetime import datetime as dt
import tensorflow

2023-04-18 09:47:56.062705: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 09:47:56.084406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 09:47:56.484059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tensorflow)

<module 'tensorflow' from '/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/__init__.py'>


In [3]:
candidates_df = pd.read_csv("./full_candidates_cateogry_id.csv")
cats = pd.read_csv("./cats.csv")


candidates_df

,category_id,title,?column?
0,af0acb21-45fc-47f1-9f4b-da567eec7e64,Consultant senior SIRH,"[{""end"": null, ""start"": ""2020-06-01"", ""title"":..."
1,d4d91d05-ec7a-4ab5-8020-b1dba5cf3e52,NaN,"[{""end"": null, ""start"": ""2020-05-01"", ""title"":..."
2,0a0f1aea-d84c-4e2e-bbed-7198b0c21729,Architecte sous-systèmes,"[{""end"": ""2022-10-31"", ""start"": ""2021-11-29"", ..."
3,abe506d3-adba-4655-847d-561813f755e3,NaN,"[{""end"": null, ""start"": ""2018-01-01"", ""title"":..."
4,083a9a77-3cfc-45af-a871-7c2f0d45355c,Data scientist,"[{""end"": null, ""start"": ""2020-03-30"", ""title"":..."
...,...,...,...
39564,d4d91d05-ec7a-4ab5-8020-b1dba5cf3e52,Talent Acquisition Manager chez CANAL+ INTERNA...,"[{""end"": null, ""start"": ""2021-06-28"", ""title"":..."
39565,cb95286b-b7d0-4ad2-b2c1-4dca6705fea2,Lead Developer - BNP Paribas,"[{""end"": null, ""start"": ""2020-04-27"", ""title"":..."
39566,50a08b3b-54fa-47f9-8546-5e62b06b6b79,Technicien assistance applications chez Groupe...,"[{""end"": null, ""start"": ""2017-05-29"", ""title"":..."
39567,11e84c07-ee61-46b0-af8f-70ca8af65c42,Conducteur de travaux cadre chez Bouygues Ener...,"[{""end"": null, ""start"": ""2020-08-31"", ""title"":..."


In [4]:
res = {}
for i in cats.values.tolist():
  res[i[0]]= i[1]


In [5]:
candidates_df["Top cat"] = candidates_df["category_id"].apply(lambda x: res[x])
candidates_df["?column?"][0]

'[{"end": null, "start": "2020-06-01", "title": "Consultant senior SIRH", "company": "Sopra HR Software", "current": true, "summary": "Gestion et mise en œuvre de projets (planning, préconisations, développement, tests, recette, mise en production, etc.).\xa0\\nRéalisation des opérations de développement, et/ou maintenance, et/ou support de composants applicatifs ou techniques de systèmes d’information, dans le respect des consignes, processus, normes qualité et sécurité, et selon les charges et planning prévus.\xa0\\nPrise en charge de projets (techniques, fonctionnels, d’intégration et de maintenance).\xa0\\nAdaptation des préconisations aux problématiques du client. Accompagnement sur la mise en œuvre.\xa0\\nProcédures établies sous Script UNIX et langage SQL sur la base Oracle.\xa0\\nInstallations des nouvelles versions des logiciels (préconisations, analyses d\'impacts sur le spécifique client, etc.).", "location": "Ville de Paris, Île-de-France, France"}, {"end": "2020-06-01", "s

In [6]:
max_len = 0


In [7]:
import json
MAX_CONTENT_LENGHT = 0

def xp_symmary(xp):
  global MAX_CONTENT_LENGHT
  xp = json.loads(xp)
  res = "\n".join([row["title"] for row in xp if row["title"]])
  if len(res) > MAX_CONTENT_LENGHT:
    MAX_CONTENT_LENGHT = len(res)
  return res

candidates_df["Formated summaries"] = candidates_df["?column?"].apply(xp_symmary)


print(MAX_CONTENT_LENGHT)
candidates_df

1009


,category_id,title,?column?,Top cat,Formated summaries
0,af0acb21-45fc-47f1-9f4b-da567eec7e64,Consultant senior SIRH,"[{""end"": null, ""start"": ""2020-06-01"", ""title"":...",Ressources Humaines,Consultant senior SIRH\nConsultant en systèmes...
1,d4d91d05-ec7a-4ab5-8020-b1dba5cf3e52,NaN,"[{""end"": null, ""start"": ""2020-05-01"", ""title"":...",Ressources Humaines,"Talent manager\nTeam leader, Account manager &..."
2,0a0f1aea-d84c-4e2e-bbed-7198b0c21729,Architecte sous-systèmes,"[{""end"": ""2022-10-31"", ""start"": ""2021-11-29"", ...",Informatique,Ingenieur IVV Système\nIngenieur IVVQ\nIngénie...
3,abe506d3-adba-4655-847d-561813f755e3,NaN,"[{""end"": null, ""start"": ""2018-01-01"", ""title"":...",Commercial,Responsable partenariats & projets stratégique...
4,083a9a77-3cfc-45af-a871-7c2f0d45355c,Data scientist,"[{""end"": null, ""start"": ""2020-03-30"", ""title"":...",Informatique,Enseignant vacataire - Introduction to chatbot...
...,...,...,...,...,...
39564,d4d91d05-ec7a-4ab5-8020-b1dba5cf3e52,Talent Acquisition Manager chez CANAL+ INTERNA...,"[{""end"": null, ""start"": ""2021-06-28"", ""title"":...",Ressources Humaines,Talent Acquisition Manager\nResearcher (Financ...
39565,cb95286b-b7d0-4ad2-b2c1-4dca6705fea2,Lead Developer - BNP Paribas,"[{""end"": null, ""start"": ""2020-04-27"", ""title"":...",Informatique,Lead Developer - Digital Laboratory\nDéveloppe...
39566,50a08b3b-54fa-47f9-8546-5e62b06b6b79,Technicien assistance applications chez Groupe...,"[{""end"": null, ""start"": ""2017-05-29"", ""title"":...",Relation Client,Technicien support applicatif N2\nTECHNICIEN S...
39567,11e84c07-ee61-46b0-af8f-70ca8af65c42,Conducteur de travaux cadre chez Bouygues Ener...,"[{""end"": null, ""start"": ""2020-08-31"", ""title"":...",Autre,Conducteur de travaux cadre\nResponsable d'aff...


In [8]:
candidates_df.size

197845

In [9]:
candidates_df = candidates_df[candidates_df["Formated summaries"].apply(lambda x: x != "")]
candidates_df.size

197575

In [10]:
from sklearn import preprocessing
from keras.utils import to_categorical
import pickle


X = candidates_df["Formated summaries"]
Y = candidates_df["Top cat"]
label_encoder = preprocessing.LabelEncoder()

label_encoder.fit(Y)

filehandler = open(b"label_encoder.pkl","wb")
pickle.dump(label_encoder,filehandler)

Y = label_encoder.transform(Y)
Y = to_categorical(Y)

In [11]:
Y.shape

(39515, 10)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pickle

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

filehandler = open(b"tokenizer.pkl","wb")
pickle.dump(tokenizer,filehandler)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = MAX_CONTENT_LENGHT

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)



In [14]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100
def create_model(learning_rate=0.001, dropout=0.4):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                               output_dim=embedding_dim, 
                               input_length=maxlen))
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.Dropout(dropout))

    model.add(layers.Dense(Y.shape[1], activation='softmax'))
    model.compile(optimizer='adam',
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])
    return model

model = create_model()


2023-04-18 09:47:58.850406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 09:47:58.859937: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 09:47:58.860088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path = '{}_my_model.best.hdf5'.format('xray_class')

#'weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only= True)
early = EarlyStopping(monitor= 'val_loss', mode= 'min', patience= 100)
callbacks_list = [checkpoint, early]

In [16]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [17]:
cnn_model = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks= callbacks_list
)

Epoch 1/10


2023-04-18 09:47:59.785940: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f80cc020340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-18 09:47:59.785974: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-04-18 09:47:59.790269: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-18 09:48:00.339552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-04-18 09:48:00.408082: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


987/988 [============================>.] - ETA: 0s - loss: 0.7270 - accuracy: 0.7943
Epoch 1: val_loss improved from inf to 0.31023, saving model to xray_class_my_model.best.hdf5
988/988 [==============================] - 16s 15ms/step - loss: 0.7265 - accuracy: 0.7944 - val_loss: 0.3102 - val_accuracy: 0.9113
Epoch 2/10
976/988 [============================>.] - ETA: 0s - loss: 0.2779 - accuracy: 0.9198
Epoch 2: val_loss improved from 0.31023 to 0.22126, saving model to xray_class_my_model.best.hdf5
988/988 [==============================] - 4s 4ms/step - loss: 0.2778 - accuracy: 0.9197 - val_loss: 0.2213 - val_accuracy: 0.9345
Epoch 3/10
978/988 [============================>.] - ETA: 0s - loss: 0.2217 - accuracy: 0.9339
Epoch 3: val_loss improved from 0.22126 to 0.19656, saving model to xray_class_my_model.best.hdf5
988/988 [==============================] - 3s 3ms/step - loss: 0.2209 - accuracy: 0.9341 - val_loss: 0.1966 - val_accuracy: 0.9408
Epoch 4/10
511/988 [==============>...

KeyboardInterrupt: 

In [ ]:
y_test.shape

(7903, 10)

In [ ]:
predictions = model.predict(x_test)

print(x_test.shape)
texts = tokenizer.sequences_to_texts(x_test)
index = 0
for p, result in zip(predictions, y_test):
    prediction = np.argmax(p)
    res = np.argmax(result) 

    if res != prediction:
        titles = texts[index]
        category = label_encoder.inverse_transform([prediction])[0]
        res = label_encoder.inverse_transform([res])[0]
        taux = np.max(p)

        print(f"{taux}: {titles} -> predicted: {category} | res: {res}")

    index += 1

247/247 [==============================] - 0s 783us/step
(7903, 1009)
0.5183098316192627: engineering director engineering manager engineering program manager lead engineering manager founder executive director executive director -> predicted: Commercial | res: Informatique
0.8706610798835754: consultante sirh sap success chef de projet rh chargée de recrutement -> predicted: Ressources Humaines | res: Relation Client
0.8434461951255798: chef de projet digital intervenant recruteur e merchandiser chef de projet seo junior assistante webmarketing et commerciale -> predicted: Marketing | res: Produit / Projet
0.9876543879508972: operations manager customer sucess manager assistante achats customer success manager development -> predicted: Relation Client | res: Commercial
0.5387092232704163: consultante fonctionnelle sirh oracle hcm cloud consultante technico fonctionnelle sirh oracle hcm cloud chargée d'applications sirh consultante sirh -> predicted: Ressources Humaines | res: Relation

In [ ]:
model = create_model(0.0001, 0.3)
model.load_weights(weight_path)
weight_path = '{}_my_model_improved.best.hdf5'.format('xray_class')

#'weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only= True)
early = EarlyStopping(monitor= 'val_loss', mode= 'min', patience= 100)
callbacks_list = [checkpoint, early]
cnn_model = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    validation_data=(x_val, y_val),
    epochs=5,
    callbacks= callbacks_list
)

NameError: name 'x_val' is not defined